# Comparing python tables

Pandas vs great_tables

# Libs

In [52]:
import numpy as np
import pandas as pd

# import geopandas as gpd
from io import BytesIO
import base64
from IPython.core.display import HTML

# from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt

# from mizani.formatters import date_format
# from mizani.breaks import date_breaks
# from scipy.stats import circmean
import pprint
import warnings

from great_tables import GT, html


warnings.filterwarnings("ignore")

# Data

In [14]:
df = pd.read_csv("data/df_nov.csv").assign(
    date=lambda x: pd.to_datetime(x["date"]),
    hour=lambda x: pd.to_datetime(x["hour"], format="%H:%M:%S").dt.hour,
)

df_oct = pd.read_csv("data/df_oct.csv").assign(
    date=lambda x: pd.to_datetime(x["date"]),
    hour=lambda x: pd.to_datetime(x["hour"], format="%H:%M:%S").dt.hour,
)

In [15]:
df_stations = pd.read_csv("data/stations.csv").rename(
    {"linea": "line", "estacion": "station"}, axis=1
)

In [16]:
rename_stations = {
    "Flores": "San Jose De Flores",
    "Saenz Peña ": "Saenz Peña",
    "Callao.b": "Callao",
    "Retiro E": "Retiro",
    "Independencia.h": "Independencia",
    "Pueyrredon.d": "Pueyrredon",
    "General Belgrano": "Belgrano",
    "Rosas": "Juan Manuel De Rosas",
    "Patricios": "Parque Patricios",
    "Mariano Moreno": "Moreno",
}

df_passengers_station = (
    df.groupby(["line", "color", "station"], as_index=False)
    .agg(pax_total=("pax_total", "sum"))
    .assign(station=lambda x: x["station"].str.title())
    .replace(rename_stations)
    .merge(df_stations, on=["line", "station"])
)

In [38]:
py_table_data = (
    df.groupby(["line", "color"], as_index=False)
    .agg(most_used_station=("station", pd.Series.mode), pax_total=("pax_total", "sum"))
    .merge(
        df_oct.groupby("line", as_index=False).agg(pax_total_oct=("pax_total", "sum")),
        on="line",
        how="left",
    )
    .assign(
        variation=lambda x: (x["pax_total"] / x["pax_total_oct"] - 1),
        # pax_total = lambda x: [str(round(i/1000000,2))+'M' for i in x['pax_total']]
    )
    .drop("color", axis=1)
)

In [73]:
custom_style = [
    {
        "selector": "caption",
        "props": [
            ("text-align", "left"),
            ("font-size", "135%"),
            ("font-weight", "bold"),
            ("width", "100%")
        ],
    },
    {
        "selector": "th",
        "props": "text-align : center; background-color: white; color: black; font-size: 18; border-bottom: 1pt solid lightgrey",
    },
    {"selector": "", "props": [("border", "1px solid lightgrey"), ("width", "100%")]},
]

title = "A summary of the use of the subway in the city of Buenos Aires, Argentina"
subtitle = "Analysis period: November 2021"

source = "<br>Own elaboration based on data from the Open Data Portal from the city of Buenos Aires</br>"

footer_map = "% of passengers per station in relation to the total number of passengers on that line. For the cuts, the quantiles of the distribution were used. It is observed that Line C presented a very high use in the headwaters, while in the rest of the lines the use was more distributed."

footer_variation = "% Variation in relation to October 2021."

footer_clock = "The color line represents the mean hour, considering the circular distribution of the hour variable."

footer_moments_day = "Percentage per line: Morning = 5 a 11 hs, Afternoon = 12 a 17 hs, Night = 18 a 23 hs"

In [79]:
pandas_table = (
    py_table_data.style.format(precision=2)
    .background_gradient(axis=0, cmap='Blues')
    .hide(axis=0)
    .set_table_styles(custom_style)
    .set_caption(
        f"""
    <h1><span style="color: darkblue">{title}</span><br></h1>
    <span style="color: black">{subtitle}</span><br><br> 
    """
    )
      .format(
    formatter={
      'variation': '{:,.2%}'.format
    }
  )
)
pandas_table

line,most_used_station,pax_total,pax_total_oct,variation
A,San Pedrito,2812536,2446385,14.97%
B,Federico Lacroze,3476972,3089913,12.53%
C,Constitucion,2129704,1782059,19.51%
D,Congreso de Tucuman,2995373,2684586,11.58%
E,Retiro E,1120315,969954,15.50%
H,Santa Fe,1515556,1387583,9.22%


In [81]:
gt_table = (
    GT(py_table_data)
    .fmt_integer(["pax_total", "pax_total_oct"], locale="es")
    .fmt_percent(["variation"])
    .cols_label(
        line="Line",
        most_used_station="Most used station",
        pax_total="Passengers (Nov)",
        pax_total_oct="Passengers (Oct)",
        variation="% Variation",
    )
    .data_color(columns = ['pax_total','pax_total_oct','variation'], palette=['white','blue'] )
    .tab_header(html(
        f"""
    <h1><span style="color: darkblue">{title}</span><br></h1>
    <span style="color: black">{subtitle}</span><br><br> 
    """
    ))
)
gt_table

GT(_tbl_data=  line    most_used_station  pax_total  pax_total_oct  variation
0    A          San Pedrito    2812536        2446385   0.149670
1    B     Federico Lacroze    3476972        3089913   0.125265
2    C         Constitucion    2129704        1782059   0.195081
3    D  Congreso de Tucuman    2995373        2684586   0.115767
4    E             Retiro E    1120315         969954   0.155019
5    H             Santa Fe    1515556        1387583   0.092227, _body=<great_tables._gt_data.Body object at 0x285806e30>, _boxhead=Boxhead([ColInfo(var='line', type=<ColInfoTypeEnum.default: 1>, column_label='Line', column_align='left', column_width=None), ColInfo(var='most_used_station', type=<ColInfoTypeEnum.default: 1>, column_label='Most used station', column_align='left', column_width=None), ColInfo(var='pax_total', type=<ColInfoTypeEnum.default: 1>, column_label='Passengers (Nov)', column_align='right', column_width=None), ColInfo(var='pax_total_oct', type=<ColInfoTypeEnum.default: 1>, column_label='Passengers (Oct)', column_align='right', column_width=None), ColInfo(var='variation', type=<ColInfoTypeEnum.default: 1>, column_label='% Variation', column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title=<great_tables._text.Text object at 0x285804ca0>, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#4848ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#0000ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9292ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#3434ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffffff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d4d4ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total_oct', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, tra

# Comparación

In [82]:
display(pandas_table)
display(gt_table)

line,most_used_station,pax_total,pax_total_oct,variation
A,San Pedrito,2812536,2446385,14.97%
B,Federico Lacroze,3476972,3089913,12.53%
C,Constitucion,2129704,1782059,19.51%
D,Congreso de Tucuman,2995373,2684586,11.58%
E,Retiro E,1120315,969954,15.50%
H,Santa Fe,1515556,1387583,9.22%


GT(_tbl_data=  line    most_used_station  pax_total  pax_total_oct  variation
0    A          San Pedrito    2812536        2446385   0.149670
1    B     Federico Lacroze    3476972        3089913   0.125265
2    C         Constitucion    2129704        1782059   0.195081
3    D  Congreso de Tucuman    2995373        2684586   0.115767
4    E             Retiro E    1120315         969954   0.155019
5    H             Santa Fe    1515556        1387583   0.092227, _body=<great_tables._gt_data.Body object at 0x285806e30>, _boxhead=Boxhead([ColInfo(var='line', type=<ColInfoTypeEnum.default: 1>, column_label='Line', column_align='left', column_width=None), ColInfo(var='most_used_station', type=<ColInfoTypeEnum.default: 1>, column_label='Most used station', column_align='left', column_width=None), ColInfo(var='pax_total', type=<ColInfoTypeEnum.default: 1>, column_label='Passengers (Nov)', column_align='right', column_width=None), ColInfo(var='pax_total_oct', type=<ColInfoTypeEnum.default: 1>, column_label='Passengers (Oct)', column_align='right', column_width=None), ColInfo(var='variation', type=<ColInfoTypeEnum.default: 1>, column_label='% Variation', column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title=<great_tables._text.Text object at 0x285804ca0>, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#4848ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#0000ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9292ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#3434ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffffff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d4d4ff')]), StyleInfo(locname='data', locnum=5, grpname=None, colname='pax_total_oct', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, tra